<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Antonio Macedo Ferrite, 21.00663-6' #@param {type:"string"}






In [238]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

In [237]:
import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [136]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [239]:
sonder_hosts = df[df['host_name'].str.contains('Sonder', na=False)]
print(sonder_hosts['host_name'].unique())

['Sonder' 'Sonder (NYC)']


In [240]:
top_hosts = df['host_id'].value_counts().head(5)
for x in top_hosts.index:
  print(df[df['host_id'] == x].host_name.unique())
  print(top_hosts[x])

['Sonder (NYC)']
7
['Stanley']
4
['Kara']
4
['Hiroki']
4
['Leonard']
2


# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [241]:
host_counts = df['host_id'].value_counts()
multi_host = host_counts[host_counts > 1].index
single_host = host_counts[host_counts == 1].index
preco_mais_de_um_imovel = df[df['host_id'].isin(multi_host)]['price'].mean()
preco_mais_um_imovel = df[df['host_id'].isin(single_host)]['price'].mean()

In [242]:
preco_mais_de_um_imovel

np.float64(218.1627906976744)

In [243]:
preco_mais_um_imovel

np.float64(150.34954007884363)

In [244]:
...

print( f'Diferença de preços: USD {preco_mais_de_um_imovel - preco_mais_um_imovel:.2f}')

Diferença de preços: USD 67.81


# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [245]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
manhattan_df = df[df['neighbourhood_group'] == 'Manhattan']
manhattan_outliers = manhattan_df[(manhattan_df['price'] < lower_bound) | (manhattan_df['price'] > upper_bound)]
perc_Manhattan = (len(manhattan_outliers) / len(manhattan_df)) * 100
print(f'{perc_Manhattan:.2f}%')


10.05%


In [246]:
...

print( f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')


Percentuais de outliers de preços, Manhattan: 10.05 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [247]:

def categorize_description(name):
    name = str(name).lower()  # Convert to lowercase for case-insensitive matching
    if 'beautiful' in name:
        return 'beautiful'
    elif 'sunny' in name:
        return 'sunny'
    elif 'spacious' in name:
        return 'spacious'
    elif 'heart' in name:
        return 'heart'
    else:
        return 'others'

df['word'] = df['name'].apply(categorize_description)

# Group data by the 'word' column
grouped_data = df.groupby('word')

# Calculate the count and average price for each group
result = grouped_data.agg({'price': ['count', 'mean']})
print(result.max())
print(result.min())
result

price  count    624.000000
       mean     166.388889
dtype: float64
price  count     36.000000
       mean     128.516667
dtype: float64


price            
          count        mean
word                       
beautiful    41  137.926829
heart        36  166.388889
others      624  157.886218
spacious     60  128.516667
sunny        43  137.674419

# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [248]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [261]:
merged_df = pd.merge(df, df_taxes, on='id', how='left')
merged_df['value'] = merged_df['value'].fillna(0)
average_taxes_by_group = merged_df.groupby('neighbourhood_group')['value'].mean()
average_taxes_by_group


neighbourhood_group
Bronx            476.230769
Brooklyn         565.202512
Manhattan        590.240997
Queens           559.878981
Staten Island    802.538462
Name: value, dtype: float64

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [257]:

df_report = df[['neighbourhood_group', 'neighbourhood', 'price']].copy()

df_report = df_report.sort_values(['neighbourhood_group', 'neighbourhood'])

df_report = df_report.reset_index(drop=True)

avg_prices = df_report.groupby('neighbourhood_group')['price'].mean()

page_id = 1
line_id = 1
report_data = []

for group_name, group_data in df_report.groupby('neighbourhood_group'):
    line_id = 1
    for index, row in group_data.iterrows():
        report_data.append([row['neighbourhood_group'], row['neighbourhood'], row['price'], page_id, line_id])
        line_id += 1
        if line_id > 30:
            page_id += 1
            line_id = 1
    report_data.append([group_name, 'mean', avg_prices[group_name], page_id, line_id])
    page_id += 1

df_report = pd.DataFrame(report_data, columns=['neighbourhood_group', 'neighbourhood', 'price', 'page_id', 'line_id'])

df_report


neighbourhood_group      neighbourhood       price  page_id  line_id
0                 Bronx            Belmont   29.000000        1        1
1                 Bronx  Claremont Village   67.000000        1        2
2                 Bronx          Concourse   40.000000        1        3
3                 Bronx           Edenwald   90.000000        1        4
4                 Bronx           Edenwald   80.000000        1        5
..                  ...                ...         ...      ...      ...
804       Staten Island        South Beach  100.000000       30        4
805       Staten Island         St. George  130.000000       30        5
806       Staten Island          Todt Hill  135.000000       30        6
807       Staten Island      Tompkinsville   59.000000       30        7
808       Staten Island               mean   89.428571       30        8

[809 rows x 5 columns]

In [258]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
799              Queens       Woodside  120.000000       29       19
800              Queens           mean   92.227848       29       20
801       Staten Island  Arden Heights   83.000000       30        1
802       Staten Island  Randall Manor   99.000000       30        2
803       Staten Island    South Beach   20.000000       30        3
804       Staten Island    South Beach  100.000000       30        4
805       Staten Island     St. George  130.000000       30        5
806       Staten Island      Todt Hill  135.000000       30        6
807       Staten Island  Tompkinsville   59.000000       30        7
808       Staten Island           mean   89.428571       30        8

In [259]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

A soma dos preços das páginas 1, 24, 25 e 28 é: 12633


#### Aqui um *sample* do report.

In [260]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
799              Queens       Woodside  120.000000       29       19
800              Queens           mean   92.227848       29       20
801       Staten Island  Arden Heights   83.000000       30        1
802       Staten Island  Randall Manor   99.000000       30        2
803       Staten Island    South Beach   20.000000       30        3
804       Staten Island    South Beach  100.000000       30        4
805       Staten Island     St. George  130.000000       30        5
806       Staten Island      Todt Hill  135.000000       30        6
807       Staten Island  Tompkinsville   59.000000       30        7
808       Staten Island           mean   89.428571       30        8

1 - Nos usamos de Data Loading para pegar os dados
2 - Usamos de Data Cleaning para verificar os outliers na Questão 3
3 - Usamos de Data Transformation para modificar o df e adicionar colunas elas na Questão 5 e 6
4 - Usamos de Data Reduction para achar valores deejados na Questão 1, em vez de pegar todos
5 - Usamos de Data Integration para unir dois datasets na questão 6
6 - Usamos Feature Engineering para criar colunas desejadas em um data set